In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.compose import make_column_transformer, ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, StandardScaler, OneHotEncoder

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor, StackingRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

import lightgbm as lgb

In [ ]:
train_df = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/train.csv")
test_df = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/test.csv")

In [ ]:
print(train_df.head())

In [ ]:
train_df.info()

81 Columns; 80 features and 1 Target (SalePrice). Our dtypes are a mix of float64(3), int64(35), and object(43). Sale Price is the last variable in our dataset which makes it easy to partition for our model building. 

We have null data present in some of our columns including: LotFrontage, Alley, MasVnrType, MasVnrArea, BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinType2, Electrical, FireplaceQu, GarageType, GarageYrBlt, GarageFinish, GarageQual, GarageCond, PoolQC, Fence, MiscFeature

In [ ]:
train_df.describe()

We don't need id as it will not provide much insight to our project, but should partition our numerical and categorical data

In [ ]:
train_df.dtypes[train_df.dtypes != 'object']

In [ ]:
train_df.dtypes[train_df.dtypes == 'object']

Taking out our outlier variables.

In [ ]:
values = [598, 955, 935, 1299, 250, 314, 336, 707, 379, 1183, 692, 186, 441, 186, 524, 739, 636, 1062, 1191, 496, 198, 1338]

In [ ]:
train_df = train_df[train_df.Id.isin(values) == False]

Now let's fill null data, starting with categorical data (object dtype)

In [ ]:
pd.DataFrame(train_df.isnull().sum().sort_values(ascending=False).head(20))

Skip our numerical variables: LotFrontage, GarageYrBlt, MasVnrArea fot now. Will likely end up dropping GarageYrBuilt as well. 

In [ ]:
train_df['PoolQC'].unique()

In [ ]:
train_df.fillna({'PoolQC':'None'}, inplace=True)
test_df.fillna({'PoolQC':'None'}, inplace=True)

In [ ]:
train_df['MiscFeature'].unique()

In [ ]:
train_df.fillna({'MiscFeature': 'None'}, inplace=True)
test_df.fillna({'MiscFeature': 'None'}, inplace=True)

In [ ]:
train_df['Alley'].unique()

In [ ]:
train_df.fillna({'Alley':'None'}, inplace=True)
test_df.fillna({'Alley':'None'}, inplace=True)

In [ ]:
train_df['Fence'].unique()

In [ ]:
train_df.fillna({'Fence':'None'}, inplace=True)
test_df.fillna({'Fence':'None'}, inplace=True)

In [ ]:
train_df['MasVnrType'].unique()

In [ ]:
train_df.fillna({'MasVnrType':'None'}, inplace=True)
test_df.fillna({'MasVnrType':'None'}, inplace=True)

In [ ]:
train_df['FireplaceQu'].unique()

In [ ]:
train_df.fillna({'FireplaceQu':'None'}, inplace=True)
test_df.fillna({'FireplaceQu':'None'}, inplace=True)

Lets take care of our garage categorical variables (Cond, Type, Finish, Qual) at the same time.

In [ ]:
train_df.fillna({'GarageCond':'None', 'GarageType':'None', 'GarageFinish':'None', 'GarageQual':'None'}, inplace=True)
test_df.fillna({'GarageCond':'None', 'GarageType':'None', 'GarageFinish':'None', 'GarageQual':'None'}, inplace=True)

We can do the same for our Basement variables (FinType1, FinType2, Exposure, Qual, Cond)

In [ ]:
train_df.fillna({'BsmtFinType1':'Unf', 'BsmtFinType2':'Unf', 'BsmtExposure':'No', 'BsmtQual':'None', 'BsmtCond':'None'}, inplace=True)
test_df.fillna({'BsmtFinType1':'Unf', 'BsmtFinType2':'Unf', 'BsmtExposure':'No', 'BsmtQual':'None', 'BsmtCond':'None'}, inplace=True)

In [ ]:
train_df['Electrical'].unique()

In [ ]:
train_df.fillna({'Electrical':'SBrkr'}, inplace=True)
test_df.fillna({'Electrical':'SBrkr'}, inplace=True)

Now lets fix our nulls for our Numerical Data.

MasVnrArea can be filled with 0, since a null would likely mean that there is no area for masonery veneer

In [ ]:
train_df.fillna({'MasVnrArea':0}, inplace=True)
test_df.fillna({'MasVnrArea':0}, inplace=True)

Lot Frontage can also be filled with 0, since a null likely means that their is no front lot

In [ ]:
train_df.fillna({'LotFrontage':0}, inplace=True)
test_df.fillna({'LotFrontage':0}, inplace=True)

We have two options for GarageYrBuilt: drop the column entirely or replace null with the YearBuilt value. A null here likely means there is no garage at all- we can do some data scoping first to make sure. We can also check to see if there is any difference in GarageYrBuilt and YrBuilt using a scatterplot. There should be very few non-linear outcomes for this since garages are typically built at the same time as the house.

In [ ]:
plt.scatter(data=train_df, x='YearBuilt', y='SalePrice')
plt.show()

Couldn't build a scatter for garage year built but we can look at correlation to our Year Built column

In [ ]:
train_df['GarageYrBlt'].corr(train_df['YearBuilt'])

Strong positive correlation of 83%, with close to 80 null values for GarageYrBuilt that would not be present in Year Built. We can drop this column and be relatively fine. 

In [ ]:
train_df = train_df.drop('GarageYrBlt', axis=1)
test_df = test_df.drop('GarageYrBlt', axis=1)

In [ ]:
pd.DataFrame(train_df.isnull().sum().sort_values(ascending=False).head(20))

Now that we've cleared out the null values, we can do some feature engineering.

Important feature data to keep in mind: MsSubClass is a categorical variable recorded as int64, OverallQual & OverallCond are ordinal variables recorded as int64, and we have some other ordinal variables we could one-hot encode to start and see if ordinal encoding would improve score later; this is due to our other ordinal variables using drastically different "ordering" variables.

In [ ]:
train_df['HouseAge'] = train_df['YrSold'] - train_df['YearBuilt']
test_df['HouseAge'] = test_df['YrSold'] - test_df['YearBuilt']

In [ ]:
train_df['RemodelAge'] = train_df['YrSold'] - train_df['YearRemodAdd']
test_df['RemodelAge'] = test_df['YrSold'] - test_df['YearRemodAdd']

In [ ]:
train_df['total_sf'] = train_df['1stFlrSF'] + train_df['2ndFlrSF'] + train_df['TotalBsmtSF']
test_df['total_sf'] = test_df['1stFlrSF'] + test_df['2ndFlrSF'] + test_df['TotalBsmtSF']

In [ ]:
#train_df['totalarea'] = train_df['GrLivArea'] + train_df['TotalBsmtSF']
#test_df['totalarea'] = test_df['GrLivArea'] + test_df['TotalBsmtSF']

In [ ]:
train_df['total_bath'] = train_df['FullBath'] + train_df['BsmtFullBath'] + 0.5 * (train_df['HalfBath']) + 0.5 * (train_df['BsmtHalfBath'])
test_df['total_bath'] = test_df['FullBath'] + test_df['BsmtFullBath'] + 0.5 * (test_df['HalfBath']) + 0.5 * (test_df['BsmtHalfBath'])

In [ ]:
train_df['TotalPorchSF'] = train_df['WoodDeckSF'] + train_df['OpenPorchSF'] + train_df['EnclosedPorch'] + train_df['3SsnPorch'] + train_df['ScreenPorch']
test_df['TotalPorchSF'] = test_df['WoodDeckSF'] + test_df['OpenPorchSF'] + test_df['EnclosedPorch'] + test_df['3SsnPorch'] + test_df['ScreenPorch']

Also need to drop ID because it would only cause confusion for our model. 

In [ ]:
train_df = train_df.drop('Id', axis=1)


In [ ]:
train_df = train_df.drop(columns=['PoolQC', 'MiscFeature', 'Alley', 'Fence', 'GarageCond', 'BsmtFinType2'], axis=1)
test_df = test_df.drop(columns=['PoolQC', 'MiscFeature', 'Alley', 'Fence', 'GarageCond', 'BsmtFinType2'], axis=1)

In [ ]:
train_df = train_df.drop(columns=['PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'LowQualFinSF', 'MSSubClass', 'TotalBsmtSF', 'GrLivArea', 'YearBuilt', 'YearRemodAdd', '1stFlrSF', '2ndFlrSF', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtFullBath', 'FullBath', 'BsmtHalfBath', 'HalfBath', 'OpenPorchSF', '3SsnPorch', 'EnclosedPorch', 'ScreenPorch','WoodDeckSF'], axis=1)
test_df = test_df.drop(columns=['PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'LowQualFinSF', 'MSSubClass', 'TotalBsmtSF', 'GrLivArea', 'YearBuilt', 'YearRemodAdd', '1stFlrSF', '2ndFlrSF', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtFullBath', 'FullBath', 'BsmtHalfBath', 'HalfBath', 'OpenPorchSF', '3SsnPorch', 'EnclosedPorch', 'ScreenPorch','WoodDeckSF'], axis=1)

In [ ]:
train_df = train_df.drop(columns=['GarageArea'], axis=1)
test_df = test_df.drop(columns=['GarageArea'], axis=1)

In [ ]:
correlation_matrix = train_df.corr(numeric_only=True)
plt.figure(figsize=(20,12))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")

Now that we've done our feature engineering we can start building our preprocessing and build our ML model. 

Again our Ordinal Columns are: OverallQual & OverallCond - we have other ordinal data that can used with a second OrdinalEncoder if we think it will improve our model accuracy

Our Categorical Columns are: MSSubClass, MSZoning, Street, Alley, LotShape, LandContour, Utilities, LotConfig, LandSlope, Neighborhood, Condition1, Condition2, BldgType, HouseStyle, RoofStyle, RoofMatl, Exterior1st, Exterior2nd, MasVnrType, ExterQual, ExterCond, Foundation, BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinType2, Heating, HeatingQC, CentralAir, Electrical, KitchenQual, Functional, FirePlaceQu, GarageType, GarageFinish, GarageQual, GarageCond, PavedDrive, PoolQC, Fence, MiscFeature, SaleType, SaleCondition

And our Numerical Columns are: LotFrontage, LotArea, YearBuilt, YearRemodAdd, MasVnrArea, BsmtFinSF1, BsmtFinSF2, BsmtUnfSF, TotalBsmtSF, 1stFlrSF, 2ndFlrSF, LowQualFinSF, GrLivArea, BsmtFullBath, BsmtHalfBath, FullBath, HalfBath, Bedroom, Kitchen, TotRmsAbvGrd, Fireplaces, GarageYrBlt, GarageCars, GarageArea, WoodDeckSF, OpenPorchSF, EnclosedPorch, 3SsnPorch, PoolArea, MiscVal, MoSold, YrSold, HouseAge, RemodelAge, total_sf, total_bath, TotalPorchSF

In [ ]:
train_df['SalePrice'] = np.log1p(train_df['SalePrice'])

In [ ]:
ord_cols = ['BsmtQual',  'BsmtFinType1',  'CentralAir',  'Functional', 'OverallQual', 'OverallCond','PavedDrive', 'LotShape', 'LandContour','Utilities','LandSlope',\
           'FireplaceQu', 'GarageFinish', 'GarageQual', 'ExterCond', 'KitchenQual', 'BsmtExposure', 'HeatingQC','ExterQual', 'BsmtCond']

In [ ]:
cat_cols = ['Street', 'LotConfig','Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'Exterior1st', 'Exterior2nd', \
           'MasVnrType','Foundation',  'Electrical',  'SaleType', 'MSZoning', 'SaleCondition', 'Heating', 'GarageType', 'RoofMatl']

In [ ]:
num_cols = train_df.select_dtypes(include=['int64', 'float64']).columns
num_cols = num_cols.drop(['SalePrice', 'OverallQual', 'OverallCond'])

In [ ]:
num_pipeline = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

In [ ]:
ord_pipeline = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='most_frequent')),
    ('ord', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
])

In [ ]:
cat_pipeline = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='most_frequent')),
    ('ohe', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

In [ ]:
col_trans = ColumnTransformer(transformers=[
    ('num_p', num_pipeline, num_cols),
    ('ord_p', ord_pipeline, ord_cols),
    ('cat_p', cat_pipeline, cat_cols)
], remainder='passthrough', n_jobs=-1)

In [ ]:
pipeline = Pipeline(steps=[
    ('preprocessing', col_trans)
])

In [ ]:
x = train_df.drop('SalePrice', axis=1)
y = train_df['SalePrice']

In [ ]:
x_preprocessed = pipeline.fit_transform(x)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_preprocessed, y, test_size=0.2, random_state=25)

In [ ]:
lr = LinearRegression()

In [ ]:
lr.fit(x_train, y_train)

In [ ]:
y_pred_lr = lr.predict(x_test)

In [ ]:
mean_squared_error(y_test, y_pred_lr)

In [ ]:
rfr = RandomForestRegressor(random_state=20)

In [ ]:
param_grid_rfr = {
    'max_depth': [5, 10, 15],
    'n_estimators': [100, 250, 500],
    'min_samples_split': [3, 5, 10]
}

In [ ]:
rfr_cv = GridSearchCV(rfr, param_grid_rfr, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

In [ ]:
rfr_cv.fit(x_train, y_train)

In [ ]:
np.sqrt(-1 * rfr_cv.best_score_)

In [ ]:
rfr_cv.best_params_

In [ ]:
xgb = XGBRegressor(random_state=20)

In [ ]:
param_grid_xgb = {
    'learning_rate': [0.1, 0.01, 0.001],
    'n_estimators': [300],
    'max_depth': [3],
    'min_child_weight': [1, 2, 3],
    'gamma': [0, 0.1, 0.2],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0]
}

In [ ]:
xgb_cv = GridSearchCV(xgb, param_grid_xgb, cv=3, scoring='neg_mean_squared_error', n_jobs=-1)

In [ ]:
xgb_cv.fit(x_train, y_train)

In [ ]:
np.sqrt(-1 * xgb_cv.best_score_)

In [ ]:
xgb_cv.best_params_

In [ ]:
ridge = Ridge()

In [ ]:
param_grid_ridge = {
    'alpha': [0.05, 0.1, 1, 3, 5, 10],
    'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag']
}

In [ ]:
ridge_cv = GridSearchCV(ridge, param_grid_ridge, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

In [ ]:
ridge_cv.fit(x_train, y_train)

In [ ]:
np.sqrt(-1 * ridge_cv.best_score_)

In [ ]:
ridge_cv.best_params_

Ridge has best score overall, however likely doesn't do the best against Test data.

In [ ]:
gbr = GradientBoostingRegressor()

In [ ]:
param_grid_gbr = {
    'max_depth': [12, 15, 20],
    'n_estimators': [200, 300, 1000],
    'min_samples_leaf': [10, 25, 50],
    'learning_rate': [0.001, 0.01, 0.1],
    'max_features': [0.01, 0.1, 0.7]
}

In [ ]:
gbr_cv = GridSearchCV(gbr, param_grid_gbr, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

In [ ]:
gbr_cv.fit(x_train, y_train)

In [ ]:
np.sqrt(-1 * gbr_cv.best_score_)

In [ ]:
gbr_cv.best_params_

In [ ]:
lgbm = lgb.LGBMRegressor(verbose=-1)

In [ ]:
param_grid_lgbm = {
    'boosting_type': ['gbdt', 'dart'],
    'num_leaves': [20, 30, 40],
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [100, 200, 300]
}

In [ ]:
lgbm_cv = GridSearchCV(lgbm, param_grid_lgbm, cv=3, scoring='neg_mean_squared_error', n_jobs=-1)

In [ ]:
lgbm_cv.fit(x_train, y_train)

In [ ]:
np.sqrt(-1 * lgbm_cv.best_score_)

In [ ]:
lgbm_cv.best_params_

In [ ]:
cat_boost = CatBoostRegressor(loss_function='RMSE', verbose=False)

In [ ]:
param_grid_cat = {
    'iterations': [100, 500, 1000],
    'depth': [4, 6, 8, 10],
    'learning_rate': [0.01, 0.05, 0.1, 0.5]
}

In [ ]:
cat_cv = GridSearchCV(cat_boost, param_grid_cat, cv=3, scoring='neg_mean_squared_error', n_jobs=-1)

In [ ]:
cat_cv.fit(x_train, y_train)

In [ ]:
np.sqrt(-1 * cat_cv.best_score_)

In [ ]:
cat_cv.best_params_

In [ ]:
vr = VotingRegressor([('gbr', gbr_cv.best_estimator_),
                     ('xgb', xgb_cv.best_estimator_),
                     ('ridge', ridge_cv.best_estimator_)],
                    weights=[2,3,1])

In [ ]:
vr.fit(x_train, y_train)

In [ ]:
y_pred_vr = vr.predict(x_test)

In [ ]:
mean_squared_error(y_test, y_pred_vr, squared=False)

In [ ]:
estimators = [
    ('gbr', gbr_cv.best_estimator_),
    ('xgb', xgb_cv.best_estimator_),
    ('cat', cat_cv.best_estimator_),
    ('lgb', lgbm_cv.best_estimator_),
    ('rfr', rfr_cv.best_estimator_),
]

In [ ]:
stackreg = StackingRegressor(
            estimators = estimators,
            final_estimator = vr
)

In [ ]:
stackreg.fit(x_train, y_train)

In [ ]:
y_pred_stack = stackreg.predict(x_test)

In [ ]:
mean_squared_error(y_test, y_pred_stack, squared=False)

In [ ]:
df_test_preprocess = pipeline.transform(test_df)

In [ ]:
y_stacking = np.exp(stackreg.predict(df_test_preprocess))

df_y_stacking_out = test_df[['Id']]
df_y_stacking_out['SalePrice'] = y_stacking

df_y_stacking_out.to_csv('submission.csv', index=False)